# Data wrangling and split-apply-combine

Big data is only useful if we know how to wrangle it. `pandas` helps us do that with a number of nifty functions, and `seaborn` often has great plotting functions to visualize the results. 

Please read the following:
1. Read about the principle of [split-apply-combine](https://pandas.pydata.org/pandas-docs/dev/user_guide/groupby.html)
2. Peruse the pandas [cookbook](https://pandas.pydata.org/pandas-docs/dev/user_guide/cookbook.html) for clever ways to slice, group, and query data. 
3. Read the introduction to the [`seaborn`](https://seaborn.pydata.org/tutorial/introduction) package. 

# Advanced `pandas` operations

In [ ]:
import pandas as pd

We're going to load in a very large dataset from my Arctic drainage density work. This is every watershed I analyzed for drainage density (the 'DD' column).

In [ ]:
sheds = pd.read_csv('https://raw.githubusercontent.com/jmdelvecchio/arctic-drainage-density/main/watershed_export.csv')

sheds.head()

The column values are:
- `Unnamed: 0` is the index of the larger dataframe that I cleaned up. Ignore (and I should have dropped before exporting! Oops)
- `long` is longitude of the center of the watershed
- `lat` is latitude of the center of the watershed
- `HYBAS_ID` is the code associated with the associated [Level 10 watershed in the HydrpSHEDS dataset](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_10). Each watershed in the dataset has a unique code. 
- `segment_ct` is the number of pixels of stream segments used to calculate drainage density
- `flow_acc` is the total number of pixels in the flow accumulation raster. `segment_ct` divided by `flow_acc` gives the drainage density `DD`.
- `relief` is the maximum elevation minus the minimum elevation within the watershed boundary
- `map_mean' is the *mean* mean annual precipiation value (in mm) within the watershed boundary
- `mat_mean` is the *mean* mean annual temperature value (in C) within the watershed boundary
- `ndvi_mean` is the *mean* of the *maximum* annual [normalized difference vegetation index](https://modis.gsfc.nasa.gov/data/dataprod/mod13.php) within the watershed boundary
- `glaciated` is the category of whether the watershed was in glacial boundaries during the Last Glacial Maximum, Marine Isotope Stage 6, or Not Glaciated
- `EXTENT` is the permafrost extent as defined by this [circum-Arctic map](https://nsidc.org/data/ggd318/versions/2)
- `DD` is drainage density, as defined above

*(if you want to know more you can always [read the paper](https://eartharxiv.org/repository/view/5340/)!)*

## Mini-assignment 

Practice using Google and reading documentation to tackle these data wrangling tasks of increasing difficulty!

How many watersheds of each category of permafrost extent are in this DataFrame?

In [ ]:
# your code here

What is the *mean* mean annual temperature of watersheds that are **unglaciated** and in **continuous permafrost**?

In [ ]:
# your code here

Generate a **list** of HYBAS IDs for a **random sample** of unglaciated watersheds in continuous permafrost whose relief is greater than 50 meters.

In [ ]:
# your code here

# `seaborn`

In [ ]:
import seaborn as sns

`seaborn` has excellent built-in functions to visualize multifaceted data stored in a DataFrame. In particular the `hue` keyword in most plots allows you to group categorial data together when plotting otherwise continuous data. 

In [ ]:
extent_order = ['Continuous', 'Discontinuous', 'Sporadic', 'Isolated', 'No permafrost']
sns.barplot(data=sheds, x='EXTENT', y='mat_mean', hue='glaciated', order=extent_order)

## Mini-assignment

Replicate the boxplot from my paper's supplement that shows how drainage density varies as a function of both permafrost extent and glacial history. Check out the seaborn boxplot documentation to customize the plot to be as close to mine as possible (not worrying about the statistical annotations). Bonus points if you can replicate the plot in the main figure that does some collapsing of the extent categories. 

In [ ]:
# Your plot here

# `geopandas`

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

## Tables to maps

Any dataframe with latitude and longitude data can be convered into a dataframe if we specify the geometry as the appropriate columns and the appropriate [coordinate reference system](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoDataFrame.crs.html) (lat/long data will usually be WGS84, EPSG:4326). We can use the `gpd.points_from_xy()` method to create a geodataframe from a regular old dataframe. 

In [ ]:
sheds_gdf = gpd.GeoDataFrame(
    sheds, # what dataframe are we using?
    geometry=gpd.points_from_xy(
        sheds['long'], sheds['lat']), # here we specify which columns have the long and lat
          crs='epsg:4326' # finally end specifying the WGS84 spatial reference
          ) 

Let's visualize the drainage density results on a world map.

In [ ]:
# Just grab world boundaries
world = gpd.read_file('ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')

fig, ax = plt.subplots(figsize=(5,5),dpi=300)
im = world.boundary.plot(
    color='k', edgecolor='black', zorder=0, ax=ax)

result = sheds_gdf.plot(
        ax=ax,
        column='DD',
        vmin=0.08,
        vmax=.16,
        cmap='seismic',
        s=0.25, #size of point
        legend=True,
        legend_kwds={
            'label': "what's a good colorbar label?",
            'orientation': "horizontal"
            },
            zorder=1
        )

ax.set_ylim(20,90)

ax.set_ylabel("ylabel")
ax.set_xlabel("xlabel")

Another great way to explore data is `gpd.explore()` (but it opens an interactive window, so large datasets might fail to load or load slowly, beware!)

In [ ]:
# sheds_gdf.explore(column='DD')

## Mini-assignment

Load in `CALM_data_regression.csv`. A previous student and I took some [fairly messy data](https://www2.gwu.edu/~calm/data/north.htm) on field observations of active layer thickness in permafrost, cleaned it up, and performed a regression on each year's measurements to get the **trend in active-layer thickness** (given as the `slope` column). Turn these data, which are points that have latitudes and longitudes, into a map colored by the **trend in active layer thickness** (with appropriate unit and axis labels.)

In [ ]:
# Your code here

# To make the next part easier make sure your GeoDataFrame is named `calm_gdf`
calm_gdf = 

## Spatial operations

I'd like to be able to compare two geospatial datasets. For example, it would be cool to know which of our watersheds might be close enough to the CALM sites that we can talk about how they've changed over time. 

In [ ]:
# Chained functions allow me to convert the CALM data to something that has units in meters
# And then create a 10 km buffer around the site
# And then put it back to lat/long
calm_gdf_buffer_geom = calm_gdf.to_crs('EPSG:3413').geometry.buffer(10000).to_crs('EPSG:4326')

# Create a new GeoDataFrame whose geometry is the buffer rather than the point. 
calm_buffer = gpd.GeoDataFrame(calm_gdf.copy(), geometry=calm_gdf_buffer_geom)

We're going to use the [sjoin](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html) function to spatially intersect our CALM site buffer against the watershed centroid points to find **watersheds that are close to CALM sites**

In [ ]:
join = gpd.sjoin(calm_buffer, sheds_gdf, how="inner")

join.head()

## Mini-assignment

How many **CALM** sites are near *more than one watershed*?

In [ ]:
# Your code here

Create a **list** of HYBAS IDs of **watersheds near CALM sites**

In [ ]:
# Your code here

# `pathlib`

I have written a lot of code to navigate directories with `pathlib`. You can read about `pathlib`[here](https://docs.python.org/3/library/pathlib.html) in the official documentaton or another guide [here](https://realpython.com/python-pathlib/). One of the strategies I have for working with my big datasets is that I often **store files related to a watershed in a directory named with the HYBAS ID** so I can iterate through directories as I might through a list from a DataFrame. 

For now we are just going to focus on making new directories and saving files into them. 

In [ ]:
from pathlib import Path

In pathlib, paths are represented as objects. You can create a Path object by passing a string representing the path to the constructor.

In [ ]:
# Creating a Path object for the current directory
current_directory = Path()

# Creating a Path object for a specific directory
specific_directory = Path("/path/to/your/directory")

It's easy to quickly make a new directory and then save a file into that directory

In [ ]:
# 2. Create two new directories
# The slash is an easy way to join an existing directory to a string you want to create into a new directory
directory1 = current_directory / "calm_sheds_intersections"

directory1.mkdir(exist_ok=True)  # exist_ok=True prevents an error if the directory already exists

## Mini-assignment

Using `pd.to_csv()`, (1) save a `pandas` dataframe of the resulting spatial join and then (2) open the dataframe from its new location

In [ ]:
# Your code here